In [2]:
from pathlib import Path
import os
import tarfile

root_path = Path().resolve()
data_path = root_path / 'data'

dict_path = data_path / 'raw' / 'stardict-eng_eng_main-2.4.2' / 'eng_eng_main.dict'
idx_path  = data_path / 'raw' / 'stardict-eng_eng_main-2.4.2' / 'eng_eng_main.idx'

print(f'{dict_path = }')
print(f'{idx_path  = }')

dict_path = PosixPath('/home/shane/workspace/who-am-i/data/raw/stardict-eng_eng_main-2.4.2/eng_eng_main.dict')
idx_path  = PosixPath('/home/shane/workspace/who-am-i/data/raw/stardict-eng_eng_main-2.4.2/eng_eng_main.idx')


In [3]:
# .idx cleansing

from collections import deque

with open(idx_path, 'rb') as idx_file:
    idx_file = idx_file.read()

q = deque(idx_file)
is_word = True
word = ''
article_start = 0
article_length = 0
with open(data_path/'clean'/'idx.csv', 'w') as f:
    while True:
        if is_word:
            try:
                b = q.popleft()
            except:
                break
            if b == 0:
                is_word = False
            else:
                word += chr(b)
        else:
            temp = ''
            for _ in range(4):
                temp += str(bin(q.popleft()))[2:]
            article_start = int(temp, 2)

            temp = ''
            for _ in range(4):
                temp += str(bin(q.popleft()))[2:]
            article_length = int(temp, 2)

            f.writelines(f'{word},{article_start},{article_length}\n')
            word = ''
            is_word = True

In [4]:
# .dict cleansing

from functools import reduce
from collections import deque

with open(dict_path, 'r') as f:
    dict_file = f.readlines()
    dict_file = map(lambda x: x.rstrip(), dict_file)
    dict_file = reduce(lambda a, b: a+b, dict_file)

with open(data_path/'clean'/'dict.txt', 'w') as f:
    f.writelines(dict_file)

In [14]:
import re
from tqdm import tqdm

with open(data_path/'clean'/'dict.txt', 'r') as f:
    dict_file = f.readline()
    k_open_tags = re.finditer('<k>', dict_file)
    k_close_tags = re.finditer('</k>', dict_file)

    tags = []
    while True:
        tag = next(k_open_tags, None)
        if tag is None:
            break
        tags.extend(tag.span())
        tag = next(k_close_tags, None)
        tags.extend(tag.span())
    
    tags.pop(0)
    tags.append(-1)

with open(data_path/'clean'/'dict.tsv', 'w') as f:
    for i in range(0, len(tags)-2, 4):
        word = dict_file[tags[i]:tags[i+1]]
        disc = dict_file[tags[i+2]:tags[i+3]]
        f.writelines(f'{word}\t{disc}\n')

In [16]:
with open(data_path/'clean'/'dict.tsv', 'r') as f:
    for row in f.readlines():
        word, disc = row.strip().split('\t')

        word = word.replace('&amp;', '&')
        word = word.replace('&quot;', '\"')
        word = word.replace('&apos;', '\'')
        word = word.replace('&lt;', '<')
        word = word.replace('&gt;', '>')

        disc = disc.replace('&amp;', '&')
        disc = disc.replace('&quot;', '\"')
        disc = disc.replace('&apos;', '\'')
        disc = disc.replace('&lt;', '<')
        disc = disc.replace('&gt;', '>')

        print(word, disc)

'cause <tr>kɔz</tr> conj. & adv. colloq. = BECAUSE. [abbr.]
'd <tr>̤d</tr> v. colloq. (usu. after pronouns) had, would (I'd; he'd). [abbr.]
'em <tr>əm</tr> pron. colloq. them (let 'em all come). [orig. a form of ME hem, dative and accus. 3rd pers. pl. pron.: now regarded as an abbr. of THEM]
'gainst <tr>ɡeɪnst</tr> prep. poet. = AGAINST. [abbr.]
'll <tr>l</tr> v. (usu. after pronouns) shall, will (I'll; that'll). [abbr.]
'm n. colloq. madam (in yes'm etc.).
'mongst poet. var. of AMONGST. [see AMONG]
'n conj. (also 'n') colloq. and. [abbr.]
'pon prep. archaic = UPON. [abbr.]
's <tr>z</tr> abbr. 1 is, has (he's; it's; John's; Charles's). 2 us (let's). 3 colloq. does (what's he say?).
's- prefix archaic (esp. in oaths) God's ('sblood; 'struth). [abbr.]
'strewth var. of 'STRUTH.
'struth int. (also 'strewth) colloq. a mild oath. [God's truth]
't <tr>t</tr> pron. contr. of IT(1) ('tis).
'tis <tr>tɪz</tr> archaic it is. [contr.]
'twas <tr>twɔz</tr> archaic it was. [contr.]
'tween <tr>twi:n</t

KeyboardInterrupt: 